## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2023xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [4]:
import os, sys
os.path
sys.path.insert(0, 'C:\\Users\\phili\\github\\ntv-pandas')

In [5]:
from datetime import datetime
import json
from observation import Sdataset, Analysis
import pandas as pd
import ntv_pandas

def analyse_integrite(data, fields, pr=True):
    '''analyse les relations du DataFrame définies dans le dict fields et retourne un dict avec pour chaque contrôle la
    liste des index ko. Les résultats des contrôles sont également ajoutés sous forme de champs booléens à data'''
    analyse = Analysis(data)
    dic_res = analyse.check_relationship(fields)
    data['ok'] = True
    for name, lis in dic_res.items():
        data[name] = True
        data.loc[lis, name] = False
        data['ok'] = data['ok'] & data[name]
        if pr:
            print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
    return dic_res

def add_nbre_pdc(data, name='ok', pr=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    #data[name] = data[name] & data['nbre_pdc_ok']
    if pr:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [6]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
#    'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
#log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
    #'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
    'chemin': 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
      }
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  64104


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [7]:
# complément à inclure dans le schéma de données
fields = [
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 0,5 % des points de charge sont hors itinerance

In [8]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  1893
nombre de pdc en itinerance   :  62211


In [9]:
# arborescence des champs et nombre de valeurs différentes
il = Sdataset(itinerance_init)
print(il.tree())

-1: root-derived (62211)
   0 : index (62211)
      1 : contact_operateur (132)
      2 : nom_enseigne (3842)
      3 : coordonneesXY (18164)
      4 : adresse_station (17749)
      5 : id_station_itinerance (19166)
      6 : nom_station (18190)
      7 : implantation_station (5)
      8 : nbre_pdc (53)
      9 : condition_acces (2)
      10: horaires (542)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (56625)
      13: date_maj (660)
      14: last_modified (447)


-----------------------
## 4 - Bilan initial intégrité
- plus de 30 % des lignes présentent un défaut d'intégrité

In [10]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          11170
contact_operateur - id_station_itinerance           7596
nom_enseigne - id_station_itinerance               13637
coordonneesXY - id_station_itinerance               8171
id_station_itinerance - id_pdc_itinerance           2368
nom_station - id_station_itinerance                 6776
implantation_station - id_station_itinerance        4080
nbre_pdc - id_station_itinerance                    4759
condition_acces - id_station_itinerance              786
horaires - id_station_itinerance                     953
station_deux_roues - id_station_itinerance          1483
adresse_station - coordonneesXY                     2953

nombre d'enregistrements sans erreurs :  41458
nombre d'enregistrements avec au moins une erreur :  20753
taux d'erreur :  33  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 27% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 25% le nombre de lignes erronnées

In [11]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'])

nombre de doublons pdc :  5586  soit  27  %
nombre de pdc sans doublon   :  15167


In [12]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            174
nom_enseigne - id_station_itinerance                5221
coordonneesXY - id_station_itinerance               5799
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 5502
implantation_station - id_station_itinerance        2963
nbre_pdc - id_station_itinerance                    4655
condition_acces - id_station_itinerance              782
horaires - id_station_itinerance                     937
station_deux_roues - id_station_itinerance           602
adresse_station - coordonneesXY                     2390

nombre d'enregistrements sans erreurs :  5499
nombre d'enregistrements avec au moins une erreur :  9668


-----------------------
## 6 - Séparation doublons station - date de maj
- 23% des erreurs résiduelles sont liées au mélange d'anciens et de nouveaux pdc
- la suppression des anciens pdc permet de réduire de 20% le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 39 opérateurs et sont liées à des causes multiples

In [13]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['etape3_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  2189  soit  23  %
nombre de pdc sans doublon   :  7479


In [14]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                3005
coordonneesXY - id_station_itinerance               3139
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 3072
implantation_station - id_station_itinerance        1364
nbre_pdc - id_station_itinerance                    2607
condition_acces - id_station_itinerance              336
horaires - id_station_itinerance                     167
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     2236

nombre d'enregistrements sans erreurs :  528
nombre d'enregistrements avec au moins une erreur :  6951  soit  72  %


In [15]:
# structure des données résiduelles et nombre de valeurs
il = Sdataset(itinerance_5)
print(il.tree())

-1: root-derived (6951)
   0 : index (6951)
      1 : contact_operateur (39)
      2 : nom_enseigne (1468)
      3 : coordonneesXY (2054)
      4 : adresse_station (2397)
      5 : id_station_itinerance (2344)
         13: date_maj (81)
         14: last_modified (59)
      6 : nom_station (2417)
      7 : implantation_station (5)
      8 : nbre_pdc (22)
      9 : condition_acces (2)
      10: horaires (77)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (6951)


In [16]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  7  erreurs :  14


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [17]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  6951
total des doublons à supprimer :  7775
nombre de pdc avec controles ok :  47485


In [18]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [19]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, pr=False)
log['bilan_erreurs'] = sum([len(controle) for controle in res.values()])
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [20]:
# structure des données bonnes
il = Sdataset(itinerance_ok.loc[:, mandatory])
print(il.tree())

-1: root-derived (47485)
   11: id_pdc_itinerance (47485)
      4 : id_station_itinerance (16653)
         1 : nom_enseigne (2000)
         2 : coordonneesXY (14734)
            0 : contact_operateur (116)
            3 : adresse_station (14417)
            8 : condition_acces (2)
         5 : nom_station (14817)
         6 : implantation_station (5)
         7 : nbre_pdc (53)
         9 : horaires (476)
         10: station_deux_roues (5)
         18: paiement_acte (8)
         19: paiement_autre (7)
         24: last_modified (372)
      12: puissance_nominale (85)
      13: prise_type_ef (8)
      14: prise_type_2 (8)
      15: prise_type_combo_ccs (7)
      16: prise_type_chademo (7)
      17: prise_type_autre (7)
      20: reservation (7)
      21: accessibilite_pmr (4)
      22: restriction_gabarit (106)
      23: date_maj (507)


### Indicateurs

In [21]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [22]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  69  % des écarts

kontakt@lidl.de 2197
support@alizecharge.fr 883
sav@izivia.com 729
supervision-ev.france@totalenergies.com 506
support@modulo-energies.fr 501


In [23]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, pr=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           4778
69  %


In [24]:
# stockage des indicateurs
log |= indic
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 29 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [25]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
632,5895,info@chargeguru.com,HYUNDAI JEAN CAUVIN - 06130 - 2,"[6.954255, 43.633261]","185 Route de Cannes, 06130 Grasse",FRCG0E00043,HYUNDAI JEAN CAUVIN - 06130 - 2,Voirie,5,Accès libre,24/7,false,FRCG0E000430,2023-10-04,2023-10-05T03:10:53.883000+00:00
633,5896,info@chargeguru.com,HYUNDAI JEAN CAUVIN - 06130 - 2,"[6.954255, 43.633261]","185 Route de Cannes, 06130 Grasse",FRCG0E00043,HYUNDAI JEAN CAUVIN - 06130 - 2,Voirie,5,Accès libre,24/7,false,FRCG0E000431,2023-10-04,2023-10-05T03:10:53.883000+00:00
634,5897,info@chargeguru.com,HYUNDAI JEAN CAUVIN - 06130 - 2,"[6.954255, 43.633261]","185 Route de Cannes, 06130 Grasse",FRCG0E00043,HYUNDAI JEAN CAUVIN - 06130 - 2,Voirie,5,Accès libre,24/7,false,FRCG0E000432,2023-10-04,2023-10-05T03:10:53.883000+00:00
635,5898,info@chargeguru.com,BMW MINI - H.Cormier La Rochelle - 17138 - 1,"[-1.104826, 46.179636]","51 Rue du 8 Mai 1945, 17138 Puilboreau",FRCG0E00043,BMW MINI - H.Cormier La Rochelle - 17138 - 1,Station dédiée à la recharge rapide,4,Accès libre,"Mo 09:00-19:00,Tu 09:00-19:00,We 09:00-19:00,T...",false,FRCG0E000433,2023-10-04,2023-10-05T03:10:53.883000+00:00
636,5899,info@chargeguru.com,BMW MINI - H.Cormier La Rochelle - 17138 - 1,"[-1.104826, 46.179636]","51 Rue du 8 Mai 1945, 17138 Puilboreau",FRCG0E00043,BMW MINI - H.Cormier La Rochelle - 17138 - 1,Station dédiée à la recharge rapide,4,Accès libre,"Mo 09:00-19:00,Tu 09:00-19:00,We 09:00-19:00,T...",false,FRCG0E000434,2023-10-04,2023-10-05T03:10:53.883000+00:00
637,5900,info@chargeguru.com,BMW MINI - H.Cormier La Rochelle - 17138 - 1,"[-1.104826, 46.179636]","51 Rue du 8 Mai 1945, 17138 Puilboreau",FRCG0E00043,BMW MINI - H.Cormier La Rochelle - 17138 - 1,Voirie,4,Accès libre,"Mo 09:00-19:00,Tu 09:00-19:00,We 09:00-19:00,T...",false,FRCG0E000435,2023-10-04,2023-10-05T03:10:53.883000+00:00
638,5901,info@chargeguru.com,BMW MINI - H.Cormier La Rochelle - 17138 - 1,"[-1.104826, 46.179636]","51 Rue du 8 Mai 1945, 17138 Puilboreau",FRCG0E00043,BMW MINI - H.Cormier La Rochelle - 17138 - 1,Voirie,4,Accès libre,"Mo 09:00-19:00,Tu 09:00-19:00,We 09:00-19:00,T...",false,FRCG0E000436,2023-10-04,2023-10-05T03:10:53.883000+00:00
639,5902,info@chargeguru.com,WATTEAU ET DIDIER - 69700 - 2,"[4.691648, 45.583802]","61 Rue des Pinaises, 69700 Beauvallon",FRCG0E00043,WATTEAU ET DIDIER - 69700 - 2,Voirie,5,Accès réservé,24/7,false,FRCG0E000437,2023-10-04,2023-10-05T03:10:53.883000+00:00
640,5903,info@chargeguru.com,WATTEAU ET DIDIER - 69700 - 2,"[4.691648, 45.583802]","61 Rue des Pinaises, 69700 Beauvallon",FRCG0E00043,WATTEAU ET DIDIER - 69700 - 2,Voirie,5,Accès réservé,24/7,false,FRCG0E000438,2023-10-04,2023-10-05T03:10:53.883000+00:00
641,5904,info@chargeguru.com,WATTEAU ET DIDIER - 69700 - 2,"[4.691648, 45.583802]","61 Rue des Pinaises, 69700 Beauvallon",FRCG0E00043,WATTEAU ET DIDIER - 69700 - 2,Voirie,5,Accès réservé,24/7,false,FRCG0E000439,2023-10-04,2023-10-05T03:10:53.883000+00:00


### Cohérence implantation_station - id_station
- 64 pdc sont liés à une erreur de choix d'implantation (3 stations, 1 opérateur)

In [26]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
30,248,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300611,2022-11-03,2023-10-04T08:02:46.946000+00:00
31,249,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Voirie,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300621,2022-11-03,2023-10-04T08:02:46.946000+00:00
32,250,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300631,2022-11-03,2023-10-04T08:02:46.946000+00:00
33,251,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300641,2022-11-03,2023-10-04T08:02:46.946000+00:00
324,1699,info@allego.eu,Allego Nissan Toulouse,"[1.42822, 43.650707]","31 Avenue Jean ZAY, 31200 Toulouse",FRALLEGO000170,Allego Nissan Toulouse,Station dédiée à la recharge rapide,2,Accès libre,24/7,false,FRALLEGO0001701,2023-10-04,2023-10-05T03:10:53.883000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990,59996,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550401,2023-07-26,2023-10-04T08:05:39.893000+00:00
6991,59997,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550402,2023-07-26,2023-10-04T08:05:39.893000+00:00
6992,59998,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550403,2023-07-26,2023-10-04T08:05:39.893000+00:00
6993,59999,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550404,2023-07-26,2023-10-04T08:05:39.893000+00:00


### Cohérence nom_station - id_station
- 328 pdc sont associés à une station avec un nom non cohérent

In [27]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
35,256,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643461,2022-11-03,2023-10-04T08:02:46.946000+00:00
36,257,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643471,2022-11-03,2023-10-04T08:02:46.946000+00:00
37,258,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684251,2022-11-03,2023-10-04T08:02:46.946000+00:00
38,259,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684261,2022-11-03,2023-10-04T08:02:46.946000+00:00
39,260,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.768425, 43.958425]",PARKING ELEPHANT BLEU - LES ANGLES,FR55CP30133,PARKING ELEPHANT BLEU - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334396228947525161,2022-11-03,2023-10-04T08:02:46.946000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990,59996,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550401,2023-07-26,2023-10-04T08:05:39.893000+00:00
6991,59997,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.4503666, 43.6039735]",TOULOUSE - Bd Lazare Carnot,FRTLSP31555040,TOULOUSE - Bd Lazare Carnot,Station dédiée à la recharge rapide,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550402,2023-07-26,2023-10-04T08:05:39.893000+00:00
6992,59998,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550403,2023-07-26,2023-10-04T08:05:39.893000+00:00
6993,59999,support@alizecharge.fr,CPO Alizé Liberté Public,"[1.45026, 43.6040701]",63 Bd Lazare Carnot 31000 TOULOUSE,FRTLSP31555040,TOULOUSE – Station Deux-Roues Lazare Carnot,Voirie,3,Accès libre,Mo-Su 00:00-23:59,FALSE,FRTLSE315550404,2023-07-26,2023-10-04T08:05:39.893000+00:00


### Cohérence adresse - coordonnées
- 1939 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [28]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18,112,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","400 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,24/7,FALSE,FR3R3E10000849681,2023-07-03,2023-10-04T08:04:41.333000+00:00
19,113,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","401 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,24/7,FALSE,FR3R3E10000849682,2023-07-03,2023-10-04T08:04:41.333000+00:00
20,114,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",708 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,FALSE,FR3R3E10000849691,2023-07-03,2023-10-04T08:04:41.333000+00:00
21,115,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",709 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,FALSE,FR3R3E10000849692,2023-07-03,2023-10-04T08:04:41.333000+00:00
22,116,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",713 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,FALSE,FR3R3E10000849851,2023-07-03,2023-10-04T08:04:41.333000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7474,63924,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2023-10-04T08:06:59.535000+00:00
7475,63933,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2023-10-04T08:06:58.549000+00:00
7476,63934,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2023-10-04T08:06:58.549000+00:00
7477,63935,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2023-10-04T08:06:58.549000+00:00


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [29]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
6436,59578,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-10-04T08:03:20.904000+00:00
6437,59579,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-10-04T08:03:20.904000+00:00
6438,59580,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-10-04T08:03:20.904000+00:00
6439,59581,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-10-04T08:03:20.904000+00:00


- exemple : Stations Ouest charge

In [30]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-3.577855, 48.771084]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
4371,43851,support@alizecharge.fr,Ouest Charge,"[-3.577855, 48.771084]",0 parking place du peillet 22560 TREBEURDEN,FRS22P22343001,Trebeurden-Place du Peillet,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E223430011,2023-07-26,2023-10-04T08:05:39.893000+00:00
4372,43852,support@alizecharge.fr,Ouest Charge,"[-3.577855, 48.771084]",0 parking place du peillet 22560 TREBEURDEN,FRS22P22343001,Trebeurden-Place du Peillet,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E223430012,2023-07-26,2023-10-04T08:05:39.893000+00:00
4555,44055,sav@izivia.com,OUEST CHARGE - 22,"[-3.577855, 48.771084]",PARKING PLACE DU PEILLET,FRS22PSDE22311,TREBEURDEN-PLACE DU PEILLET,Voirie,2,Accès libre,24/7,false,FRS22ESDE22311,2023-04-07,2023-10-04T08:03:31.328000+00:00
4556,44056,sav@izivia.com,OUEST CHARGE - 22,"[-3.577855, 48.771084]",PARKING PLACE DU PEILLET,FRS22PSDE22312,TREBEURDEN-PLACE DU PEILLET,Voirie,2,Accès libre,24/7,false,FRS22ESDE22312,2023-04-07,2023-10-04T08:03:31.328000+00:00


### Cohérence station - coordonnées
- 477 pdc sont associés à des stations avec plusieurs coordonnées

In [31]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
446,5605,info@chargeguru.com,BAIKOWSKI,"[6.067396, 45.936453]","1046 Route de Chaumontet, 74330 Poisy",FRCG0E00003,BAIKOWSKI,Voirie,9,Accès réservé,24/7,false,FRCG0E000030,2023-10-05,2023-10-05T03:10:53.883000+00:00
447,5606,info@chargeguru.com,BAIKOWSKI,"[6.067396, 45.936453]","1046 Route de Chaumontet, 74330 Poisy",FRCG0E00003,BAIKOWSKI,Voirie,9,Accès réservé,24/7,false,FRCG0E000036,2023-10-05,2023-10-05T03:10:53.883000+00:00
448,5607,info@chargeguru.com,BAIKOWSKI,"[6.067396, 45.936453]","1046 Route de Chaumontet, 74330 Poisy",FRCG0E00003,BAIKOWSKI,Voirie,9,Accès réservé,24/7,false,FRCG0E000037,2023-10-05,2023-10-05T03:10:53.883000+00:00
449,5608,info@chargeguru.com,Borne de recharge SATELC - 22kW,"[6.467133, 45.910459]","418 Route des Chenons, 74220 La Clusaz",FRCG0E00003,Borne de recharge SATELC - 22kW,Voirie,3,Accès libre,24/7,false,FRCG0E000039,2023-10-05,2023-10-05T03:10:53.883000+00:00
450,5609,info@chargeguru.com,Borne de recharge SATELC - 22kW,"[6.467133, 45.910459]","418 Route des Chenons, 74220 La Clusaz",FRCG0E00004,Borne de recharge SATELC - 22kW,Voirie,3,Accès libre,24/7,false,FRCG0E000040,2023-10-05,2023-10-05T03:10:53.883000+00:00
451,5610,info@chargeguru.com,Borne de recharge SATELC - 22kW,"[6.467133, 45.910459]","418 Route des Chenons, 74220 La Clusaz",FRCG0E00004,Borne de recharge SATELC - 22kW,Voirie,3,Accès libre,24/7,false,FRCG0E000041,2023-10-05,2023-10-05T03:10:53.883000+00:00
452,5611,info@chargeguru.com,Les Saisies - Borne de recharge,"[6.544075, 45.75317]","Route du Mont Bisanne, 73620 Hauteluce",FRCG0E00004,Les Saisies - Borne de recharge,Voirie,3,Accès libre,24/7,false,FRCG0E000042,2023-10-05,2023-10-05T03:10:53.883000+00:00
453,5612,info@chargeguru.com,Les Saisies - Borne de recharge,"[6.544075, 45.75317]","Route du Mont Bisanne, 73620 Hauteluce",FRCG0E00004,Les Saisies - Borne de recharge,Voirie,3,Accès libre,24/7,false,FRCG0E000043,2023-10-05,2023-10-05T03:10:53.883000+00:00
454,5613,info@chargeguru.com,Les Saisies - Borne de recharge,"[6.544075, 45.75317]","Route du Mont Bisanne, 73620 Hauteluce",FRCG0E00004,Les Saisies - Borne de recharge,Voirie,3,Accès libre,24/7,false,FRCG0E000044,2023-10-05,2023-10-05T03:10:53.883000+00:00
455,5615,info@chargeguru.com,Mairie de Hauteluce - Chef-lieu,"[6.582946, 45.749162]","Route de Hauteluce, 73620 Hauteluce",FRCG0E00004,Mairie de Hauteluce - Chef-lieu,Voirie,3,Accès libre,24/7,false,FRCG0E000046,2023-10-05,2023-10-05T03:10:53.883000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [32]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,6,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-10-04T08:00:15.369000+00:00
1,7,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-10-04T08:00:15.369000+00:00
2,8,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2023-10-04T08:00:15.369000+00:00
3,9,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-10-04T08:00:15.369000+00:00
5,11,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-10-04T08:00:15.369000+00:00
6,12,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2023-10-04T08:00:15.369000+00:00
7,13,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-10-04T08:00:15.369000+00:00
8,14,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2023-10-04T08:00:15.369000+00:00
9,15,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2023-10-04T08:00:15.369000+00:00
10,16,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-10-04T08:00:15.369000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [33]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
40,267,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2023-10-04T08:02:46.946000+00:00
41,268,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2023-10-04T08:02:46.946000+00:00
42,269,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2023-10-04T08:02:46.946000+00:00
43,270,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-10-04T08:02:46.946000+00:00
44,271,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2023-10-04T08:02:46.946000+00:00
45,272,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2023-10-04T08:02:46.946000+00:00


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [34]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### Incohérence 'nbre_pdc'
- 25% des pdc ont un champ 'nbre_pdc' mal documenté

- exemple : station avec un pdc 'de regroupement' qui évite de documenter plusieurs lignes

In [35]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRLUMEACACIAS11', 
               ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
33711,FRLUMEACACIAS11,FRLUMEACACIAS11,"[3.1485441,50.7582304]",4


- exemple : station avec plusieurs pdc mais l'id_station est identique à l'id_pdc

In [36]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[2.451322, 45.66523]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc


- exemple : station avec plusieurs pdc mais le champs a toujours une valeur de 1

In [37]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[-1.7548354193520386, 48.125067030488154]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc


### cohérence station - enseigne
- exemple de station avec plusieurs noms d'enseigne

In [38]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS27PBARREOUCHEMOMORT', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
43622,44373,pascal.lhermitte@siege27.fr,SIEGE265,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTD,2021-06-15,2023-10-04T08:02:26.105000+00:00
43623,44374,pascal.lhermitte@siege27.fr,SIEGE266,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTG,2021-06-15,2023-10-04T08:02:26.105000+00:00


In [39]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRLDLE0000125', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
27678,28319,kontakt@lidl.de,AIZENAY - Roche,"[-1.591891, 46.734313]","Rue de la Roche 121, 85190 Aizenay",FRLDLE0000125,AIZENAY - Roche,Voirie,4,Accès libre,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",false,FRLDLE00001250,2023-10-05,2023-10-05T03:10:53.883000+00:00
27679,28320,kontakt@lidl.de,AIZENAY - Roche,"[-1.591891, 46.734313]","Rue de la Roche 121, 85190 Aizenay",FRLDLE0000125,AIZENAY - Roche,Voirie,4,Accès libre,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",false,FRLDLE00001251,2023-10-05,2023-10-05T03:10:53.883000+00:00
27680,28321,kontakt@lidl.de,AIZENAY - Roche,"[-1.591891, 46.734313]","Rue de la Roche 121, 85190 Aizenay",FRLDLE0000125,AIZENAY - Roche,Voirie,4,Accès libre,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",false,FRLDLE00001252,2023-10-05,2023-10-05T03:10:53.883000+00:00
27681,28322,kontakt@lidl.de,Lidl PARTHENAY,"[-0.217602, 46.647709]","Avenue Aristide Briand 201, 79200 Parthenay",FRLDLE0000125,Lidl PARTHENAY,Voirie,4,Accès libre,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",false,FRLDLE00001253,2023-10-05,2023-10-05T03:10:53.883000+00:00
27682,28323,kontakt@lidl.de,Lidl PARTHENAY,"[-0.217602, 46.647709]","Avenue Aristide Briand 201, 79200 Parthenay",FRLDLE0000125,Lidl PARTHENAY,Voirie,4,Accès libre,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",false,FRLDLE00001254,2023-10-05,2023-10-05T03:10:53.883000+00:00
27683,28324,kontakt@lidl.de,Lidl THOUARS,"[-0.206407, 46.996375]","Avenue Emile Zola, 79100 Thouars",FRLDLE0000125,Lidl THOUARS,Voirie,4,Accès libre,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",false,FRLDLE00001255,2023-10-05,2023-10-05T03:10:53.883000+00:00
27684,28325,kontakt@lidl.de,Lidl THOUARS,"[-0.206407, 46.996375]","Avenue Emile Zola, 79100 Thouars",FRLDLE0000125,Lidl THOUARS,Voirie,4,Accès libre,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",false,FRLDLE00001256,2023-10-05,2023-10-05T03:10:53.883000+00:00
27685,28326,kontakt@lidl.de,BONNEE,"[2.38001, 47.793319]","Route de Sully, 45460 Bonnée",FRLDLE0000125,BONNEE,Voirie,2,Accès libre,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",false,FRLDLE00001257,2023-10-05,2023-10-05T03:10:53.883000+00:00
27686,28327,kontakt@lidl.de,BONNEE,"[2.38001, 47.793319]","Route de Sully, 45460 Bonnée",FRLDLE0000125,BONNEE,Voirie,2,Accès libre,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",false,FRLDLE00001258,2023-10-05,2023-10-05T03:10:53.883000+00:00
27687,28328,kontakt@lidl.de,Lidl LA CHATRE,"[1.999847, 46.582605]","Avenue d'Auvergne, 36400 La Châtre",FRLDLE0000125,Lidl LA CHATRE,Voirie,2,Accès libre,"Mo 00:00-23:59,Tu 00:00-23:59,We 00:00-23:59,T...",false,FRLDLE00001259,2023-10-05,2023-10-05T03:10:53.883000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire

In [40]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
43454,44205,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:01,False,FRS23P230200101,2023-03-09,2023-10-04T08:01:51.686000+00:00
43455,44206,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:02,False,FRS23P230200102,2023-03-09,2023-10-04T08:01:51.686000+00:00


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues

In [41]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
61196,62036,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,True,FRV75EPX17161,2023-07-06,2023-10-04T08:03:20.145000+00:00
61197,62037,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,True,FRV75EPX17162,2023-07-06,2023-10-04T08:03:20.145000+00:00
61198,62038,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,False,FRV75EPX17163,2023-07-06,2023-10-04T08:03:20.145000+00:00
61199,62039,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,False,FRV75EPX17164,2023-07-06,2023-10-04T08:03:20.145000+00:00
61200,62040,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,False,FRV75EPX17165,2023-07-06,2023-10-04T08:03:20.145000+00:00
61201,62041,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,False,FRV75EPX17166,2023-07-06,2023-10-04T08:03:20.145000+00:00
61202,62042,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,False,FRV75EPX17167,2023-07-06,2023-10-04T08:03:20.145000+00:00


### cohérence coordonnées - adresse
- exemple de plusieurs stations avec des coordonnées identiques mais des adresses différentes

In [42]:
itinerance_5.loc[itinerance_5.coordonneesXY == '[2.460441, 50.78763]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1312,24377,support@passpasselectrique.fr,Pass Pass Electrique,"[2.460441, 50.78763]",1bis Rue De La Gare 59670 Bavinchove,FRH14E59054001,Bavinchove - Gare,Parking public,2,Accès libre,24/7,false,FRH14E59054001,2023-01-27,2023-10-04T08:02:24.274000+00:00
1313,24378,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540011,2023-07-26,2023-10-04T08:05:39.893000+00:00
1314,24379,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540012,2023-07-26,2023-10-04T08:05:39.893000+00:00
1315,24380,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540013,2023-07-26,2023-10-04T08:05:39.893000+00:00
1316,24381,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540014,2023-07-26,2023-10-04T08:05:39.893000+00:00


### données d'un opérateur
- exemple Chargeguru

In [43]:
itinerance_operateur = itinerance_5_full.loc[itinerance_5_full.contact_operateur == 'info@chargeguru.com']
#itinerance_operateur.loc[itinerance_operateur['adresse_station - coordonneesXY'] == False, relations]
itinerance_operateur.loc[itinerance_operateur['nom_station - id_station_itinerance'] == False, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
411,5588,info@chargeguru.com,Office Notarial BDN,"[2.403973, 51.043618]","1 Boulevard Paul Cambon, 59240 Dunkerque",FRCG0E00000,Office Notarial BDN,Voirie,1,Accès réservé,24/7,false,FRCG0E000001,2023-10-05,2023-10-05T03:10:53.883000+00:00
412,5589,info@chargeguru.com,Zen Park Villeneuve d'Ascq - 59650 - 5,"[3.145543, 50.671322]","54 Avenue Le Notre, 59650 Villeneuve d\\'Ascq",FRCG0E00000,Zen Park Villeneuve d'Ascq - 59650 - 5,Voirie,8,Accès réservé,24/7,false,FRCG0E000004,2023-10-05,2023-10-05T03:10:53.883000+00:00
413,5590,info@chargeguru.com,Zen Park Villeneuve d'Ascq - 59650 - 5,"[3.145543, 50.671322]","54 Avenue Le Notre, 59650 Villeneuve d\\'Ascq",FRCG0E00000,Zen Park Villeneuve d'Ascq - 59650 - 5,Voirie,8,Accès réservé,24/7,false,FRCG0E000005,2023-10-05,2023-10-05T03:10:53.883000+00:00
414,5591,info@chargeguru.com,Zen Park Villeneuve d'Ascq - 59650 - 5,"[3.145543, 50.671322]","54 Avenue Le Notre, 59650 Villeneuve d\\'Ascq",FRCG0E00000,Zen Park Villeneuve d'Ascq - 59650 - 5,Voirie,8,Accès réservé,24/7,false,FRCG0E000006,2023-10-05,2023-10-05T03:10:53.883000+00:00
415,5592,info@chargeguru.com,Zen Park Villeneuve d'Ascq - 59650 - 5,"[3.145543, 50.671322]","54 Avenue Le Notre, 59650 Villeneuve d\\'Ascq",FRCG0E00000,Zen Park Villeneuve d'Ascq - 59650 - 5,Voirie,8,Accès réservé,24/7,false,FRCG0E000007,2023-10-05,2023-10-05T03:10:53.883000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,6205,info@chargeguru.com,BUREAU VALLEE - 91700 - 1,"[2.347218, 48.623376]","3 Rue de la Remise Neuve, 91700 Sainte-Geneviè...",FRCG0E00179,BUREAU VALLEE - 91700 - 1,Voirie,3,Accès libre,24/7,false,FRCG0E001793,2023-10-04,2023-10-05T03:10:53.883000+00:00
687,6206,info@chargeguru.com,BUREAU VALLEE - 91700 - 1,"[2.347218, 48.623376]","3 Rue de la Remise Neuve, 91700 Sainte-Geneviè...",FRCG0E00179,BUREAU VALLEE - 91700 - 1,Voirie,3,Accès libre,24/7,false,FRCG0E001794,2023-10-04,2023-10-05T03:10:53.883000+00:00
690,6219,info@chargeguru.com,SCALA - Volkswagen Pamiers - 09100 - 2,"[1.63066, 43.108533]","Village Automobile, All. Bugatti, 09100 Pamiers",FRCG0E00186,SCALA - Volkswagen Pamiers - 09100 - 2,Voirie,7,Accès libre,24/7,false,FRCG0E001860,2023-10-05,2023-10-05T03:10:53.883000+00:00
691,6220,info@chargeguru.com,Groupe Fabre - Audi Rodez - 12850 - 1,"[2.543264, 44.375811]","Rue des Enlumineurs, 12850 Onet-le-Château",FRCG0E00186,Groupe Fabre - Audi Rodez - 12850 - 1,Station dédiée à la recharge rapide,2,Accès libre,24/7,false,FRCG0E001863,2023-10-05,2023-10-05T03:10:53.883000+00:00


In [44]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRCG0E00052', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
5722,5924,info@chargeguru.com,BMW - ALTITUDE 69 LYON - 69009 - 1,"[4.80905507, 45.78509803]","6 Rue Joannes Carret, 69009 Lyon",FRCG0E00052,BMW - ALTITUDE 69 LYON - 69009 - 1,Voirie,13,Accès libre,24/7,false,FRCG0E000520,2023-10-05,2023-10-05T03:10:53.883000+00:00
5723,5925,info@chargeguru.com,Hôtel Berny - 92160,"[2.308095, 48.762504]","129 Avenue Aristide Briand, 92160 Antony",FRCG0E00052,Hôtel Berny - 92160,Voirie,1,Accès réservé,24/7,false,FRCG0E000526,2023-10-03,2023-10-05T03:10:53.883000+00:00


In [45]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRCG0E00012', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
5474,5676,info@chargeguru.com,BMW MINI - ISERAN AUTOMOBILES - CHAMBERY - 730...,"[5.89408, 45.598355]","Rue Alexander Fleming, 73000 Chambéry",FRCG0E00012,BMW MINI - ISERAN AUTOMOBILES - CHAMBERY - 730...,Voirie,8,Accès libre,"Mo 08:00-18:00,Tu 08:00-18:00,We 08:00-18:00,T...",false,FRCG0E000120,2023-10-04,2023-10-05T03:10:53.883000+00:00
5475,5677,info@chargeguru.com,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,"[6.266378, 46.198887]","8 Rue des Buchillons, 74100 Ville-la-Grand",FRCG0E00012,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,Station dédiée à la recharge rapide,6,Accès libre,24/7,false,FRCG0E000121,2023-10-05,2023-10-05T03:10:53.883000+00:00
5476,5678,info@chargeguru.com,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,"[6.266378, 46.198887]","8 Rue des Buchillons, 74100 Ville-la-Grand",FRCG0E00012,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,Station dédiée à la recharge rapide,6,Accès libre,24/7,false,FRCG0E000122,2023-10-05,2023-10-05T03:10:53.883000+00:00
5477,5679,info@chargeguru.com,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,"[6.266378, 46.198887]","8 Rue des Buchillons, 74100 Ville-la-Grand",FRCG0E00012,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,Station dédiée à la recharge rapide,6,Accès libre,24/7,false,FRCG0E000123,2023-10-05,2023-10-05T03:10:53.883000+00:00
5478,5680,info@chargeguru.com,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,"[6.266378, 46.198887]","8 Rue des Buchillons, 74100 Ville-la-Grand",FRCG0E00012,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,Voirie,6,Accès libre,24/7,false,FRCG0E000124,2023-10-05,2023-10-05T03:10:53.883000+00:00
5479,5681,info@chargeguru.com,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,"[6.266378, 46.198887]","8 Rue des Buchillons, 74100 Ville-la-Grand",FRCG0E00012,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,Voirie,6,Accès libre,24/7,false,FRCG0E000125,2023-10-05,2023-10-05T03:10:53.883000+00:00
5480,5682,info@chargeguru.com,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,"[6.266378, 46.198887]","8 Rue des Buchillons, 74100 Ville-la-Grand",FRCG0E00012,BMW - SALEVE AUTOMOBILES - VILLE-LA-GRAND - 74...,Voirie,6,Accès libre,24/7,false,FRCG0E000126,2023-10-05,2023-10-05T03:10:53.883000+00:00
5481,5683,info@chargeguru.com,BMW MINI - ISERAN AUTOMOBILES - CHAMBERY - 730...,"[5.89408, 45.598355]","Rue Alexander Fleming, 73000 Chambéry",FRCG0E00012,BMW MINI - ISERAN AUTOMOBILES - CHAMBERY - 730...,Station dédiée à la recharge rapide,8,Accès libre,"Mo 08:00-18:00,Tu 08:00-18:00,We 08:00-18:00,T...",false,FRCG0E000127,2023-10-04,2023-10-05T03:10:53.883000+00:00
5482,5684,info@chargeguru.com,BMW MINI - ISERAN AUTOMOBILES - CHAMBERY - 730...,"[5.89408, 45.598355]","Rue Alexander Fleming, 73000 Chambéry",FRCG0E00012,BMW MINI - ISERAN AUTOMOBILES - CHAMBERY - 730...,Station dédiée à la recharge rapide,8,Accès libre,"Mo 08:00-18:00,Tu 08:00-18:00,We 08:00-18:00,T...",false,FRCG0E000128,2023-10-04,2023-10-05T03:10:53.883000+00:00
5483,5685,info@chargeguru.com,BMW MINI - ISERAN AUTOMOBILES - CHAMBERY - 730...,"[5.89408, 45.598355]","Rue Alexander Fleming, 73000 Chambéry",FRCG0E00012,BMW MINI - ISERAN AUTOMOBILES - CHAMBERY - 730...,Station dédiée à la recharge rapide,8,Accès libre,"Mo 08:00-18:00,Tu 08:00-18:00,We 08:00-18:00,T...",false,FRCG0E000129,2023-10-04,2023-10-05T03:10:53.883000+00:00
